# СЕМИНАР 3: МЕТОД МАКСИМАЛЬНОГО ПРАВДОПОДОБИЯ

На лекции мы рассмотрели теоретические основы метода максимального правдоподобия и даже вывели несколько оценок ММП руками.

Попробуем провернуть аналогичные операции в Python


## Практический минимум

In [5]:
import numpy as np
import pandas as pd

from scipy import stats
from scipy.optimize import minimize

import seaborn as sns
import matplotlib.pyplot as plt

* Попробуем оценить параметры нормального распределения для следующей выборки:

In [8]:
x = [3.2 , 7.9, 5.4, 4.9, 6.2, 4.3]

1. Напишем ф-ю логарифма макс. правдоподобия, которая принимает на вход массив из двух параметров норм. распр-я и нашу выборку.

   Возвращает значение лог-функции правдоподобия

In [ ]:
def lnL(theta, x):
    mu = theta[0]
    s2 = np.exp(theta[1])

    x = np.array(x)
    n = x.size

    l = -0.5*n*np.log(s2) - 0.5/s2*np.sum((x-mu)**2) # лог ф-я правдоподобия для норм. распр-я

    return - l # на минус 1 домножаем в силу того, что в scipy есть только функция minimize

# Проверка, что работает
print(lnL([4, 0.2], x))

10.240554617493236


2. Оценим параметры

In [ ]:
theta_int = [0,0] # initial guess
res = minimize(lnL, theta_int, args=x)
res

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 5.353606615954055
        x: [ 5.317e+00  7.845e-01]
      nit: 14
      jac: [-1.729e-06  2.265e-06]
 hess_inv: [[ 3.527e-01  1.299e-02]
            [ 1.299e-02  3.198e-01]]
     nfev: 51
     njev: 17

In [12]:
mu, s2 = res.x
s2 = np.sqrt(np.exp(s2))
mu, s2

(5.3166660045167635, 1.480334583591595)

3. Получим тот же результат, используя встроенный метод stats.norm.fit(x)

In [13]:
mu, s = stats.norm.fit(x)
mu, s

(5.316666666666666, 1.4803340463857775)

4. Построим 95% доверительный интервал для среднего

In [14]:
res.hess_inv

array([[0.35265742, 0.0129908 ],
       [0.0129908 , 0.31978713]])

In [15]:
alpha = 0.05

z = stats.norm().ppf(1-alpha/2)

left = res.x[0] - z*np.sqrt(res.hess_inv[0,0])
right = res.x[0] + z*np.sqrt(res.hess_inv[0,0])

print(f'Доверительный интервал для среднего: [{left} ; {right}]')

Доверительный интервал для среднего: [4.152742052385886 ; 6.480589956647641]


Ниже на примере нормального распределения написан пайплайн запуска теста отношения правдоподобий

In [7]:
np.random.seed(42)

n = 100          # размер выборки
mu_true = 0      # истинное среднее
sigma = 1        # известное стандартное отклонение
alpha = 0.05     # уровень значимости

hypo_mu = 0.15   # наше предположение о параметре как исследователей

# Генерируем выборку из нормального распределения
data = np.random.normal(loc=mu_true, scale=sigma, size=n)

# Оценка параметра при H1 (альтернативная гипотеза)
# MLE для параметра `mu` при нормальном распределении с известной дисперсией – выборочное среднее
mu_hat = np.mean(data)
print('Оцененное среднее:', mu_hat)

# Вычисление логарифмов правдоподобия
logL_R = np.sum(stats.norm.logpdf(data, loc=hypo_mu, scale=sigma))     # при mu = hypo_mu (H0)
logL_UR = np.sum(stats.norm.logpdf(data, loc=mu_hat, scale=sigma))  # при mu = mu_hat (H1)

# Вычисление статистики теста: -2 * (logL_R - logL_UR)
test_statistic = -2 * (logL_R - logL_UR)
print("Статистика теста: {:.4f}".format(test_statistic))

# Вычисление критического значения для уровня значимости alpha = 0.05
critical_value = stats.chi2.ppf(1 - alpha, df=1)
print("Критическое значение: {:.4f}".format(critical_value))

# Вычисление p-value c 1 степенью свободы, ибо мы ограничили один параметр
p_value = 1 - stats.chi2.cdf(test_statistic, df=1)
print("p-value: {:.4f}".format(p_value))

# Принятие решения
if p_value < alpha:
    print(f"Отвергаем H0: данные не согласуются с предположением mu = {hypo_mu}")
else:
    print(f"Нет оснований отвергать H0: данные согласуются с предположением mu = {hypo_mu}")

Оцененное среднее: -0.10384651739409384
Статистика теста: 6.4438
Критическое значение: 3.8415
p-value: 0.0111
Отвергаем H0: данные не согласуются с предположением mu = 0.15


5. Попробуйте провести тест отношения правдоподобий в предыдущей задаче с гипотезой о том, что среднее равняется единице

In [ ]:
### YOUR CODE HERE

6. А теперь тест отношения правдоподобий с гипотезой о том, что среднее равно единице, а дисперсия двойке

In [ ]:
### YOUR CODE HERE

## Практика

* На практике для давно известных распределений никто не оценивает параметры ММП в каждой задаче, а просто используют уже известные оценки, выведенные в общем виде.

* Например, для аналитики спорта очень часто используют Пуассоновское распределение.

* Им моделируют, например, общее кол-во голов в матче, голы домашней команды, команды соперника, кол-во успешных действий конкретного игрока, кол-во красных и желтых карточек... В общем любой "подсчет" в единицу времени

* В этой практике попробуем немного погрузиться в экономику спорта

В данной задачи будем использовать данные с <https://footystats.org/> о матчах в самых известных футбольных премьер лигах за 5 последних сезонов.

Данные можно загрузить по ссылке: https://drive.google.com/drive/folders/16WZMzHd74W-O_NdUgsEPjJ9rTXywsN1t?usp=drive_link

Описание данных можно найти здесь: https://footystats.org/api/documentations/match-schedule-and-stats

### TASK 1 (OPTIONAL) 

Если есть желание попрактиковаться, попробуйте вывести оценки ММП для распределение Пуассона в общем виде

In [ ]:
### YOUR CODE OR LATEX HERE

### TASK 2: Спорт как Пуассоновская модель

* Объедините все голы, забитые во всех матчах: и домашние, и гостевые, либо возьмите готовый столбец из данных. 

* Предположив, что общее количество голов в лиге подчиняется распределению Пуассона, найдите оценку максимального правдоподобия для параметра λ (среднее количество голов за матч в лиге)

* Насколько она устойчива к изменению выборки (например, если взять только матчи одной команды или одного сезона)?

* Графические иллюстрации приветствуются, но не обязательны :)

In [ ]:
### YOUR CODE HERE

### TASK 3: Стиль игры

1. Выберите одну конкретную команду. Оцените два параметра:

* λ_attack – среднее количество голов, забиваемое этой командой за матч (используйте данные по всем матчам, где она играла).

* λ_defence – среднее количество голов, пропускаемых этой командой за матч.

2. Сравните λ_attack и λ_defence для этой команды. Что это говорит о ее стиле игры? Является ли она атакующей, оборонительной или сбалансированной?

In [ ]:
### YOUR CODE HERE

### TASK 4: Существует ли эффект домашнего поля?

* Формулировка гипотезы: Команды в среднем забивают больше голов, играя дома, чем в гостях.


1. Для выбранной команды (или для всех команд в совокупности) оцените λ_home (средние голы дома) и λ_away (средние голы в гостях).

2. Постройте 95% доверительные интервалы для λ_home и λ_away (можно использовать свойство асимптотической нормальности оценки ММП). Перекрываются ли они?

3. Проверьте гипотезу H₀: λ_home = λ_away против H₁: λ_home > λ_away (Hint: можно использовать тест отношения правдоподобия).

In [17]:
### YOUR CODE HERE